In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pushbullet.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from keras.models import load_model
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
from datetime import datetime
import pytz
from PIL import Image
from PIL import ImageEnhance
import os
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle

In [4]:
def getTime():
  IST = pytz.timezone('Asia/Kolkata')
  timeNow = datetime.now(IST)
  return timeNow

In [5]:
def imgenhance():
  image1 = Image.open('savedImage.jpg')
  curr_bri = ImageEnhance.Sharpness(image1)
  new_bri = 1.3
  img_brightened = curr_bri.enhance(new_bri)
  im1 = img_brightened.save("bright.jpg")

  image2 = Image.open('bright.jpg')
  curr_col = ImageEnhance.Color(image2)
  new_col = 1.5
  img_col = curr_col.enhance(new_col)
  im2 = img_col.save("finalImage.jpg")

# **Notification**

In [6]:
import requests
from pushbullet import Pushbullet

def push_vio():
  pb = Pushbullet('o.4QgWjylmET6zEZkDpzF5f3jRb4YNegrN')

  devices = ['ujyalL8mfoisjAZYWtbUPY']
  message = 'Alert! Suspicious activity detected\nClick on the following link'
  for device in devices:  
    # push = device.push_note('Notification', message)
    push = pb.push_link(message,'https://drive.google.com/file/d/1pQaHpPuMjTuiRFQIDvIlSeDzXqb0q2EJ/view?usp=share_link')

def push_mob():
  pb = Pushbullet('o.4QgWjylmET6zEZkDpzF5f3jRb4YNegrN')

  devices = ['ujyalL8mfoisjAZYWtbUPY']
  message = 'Alert! Suspicious activity detected\nClick on the following link'
  for device in devices:  
    # push = device.push_note('Notification', message)
    push = pb.push_link(message,'https://drive.google.com/file/d/15hB4Zhsjh2FWMJEf3nQ-YPF7q9nini7c/view?usp=share_link')

def push_wep():
  pb = Pushbullet('o.4QgWjylmET6zEZkDpzF5f3jRb4YNegrN')

  devices = ['ujyalL8mfoisjAZYWtbUPY']
  message = 'Alert! Suspicious activity detected\nClick on the following link'
  for device in devices:  
    # push = device.push_note('Notification', message)
    push = pb.push_link(message,'https://drive.google.com/file/d/1mtSvMDhCjYkgJ2SK4M0E914X7r16gTgO/view?usp=share_link')

In [7]:
V_path = "/content/drive/MyDrive/WVM/Violence/Testingvideos/V_19.mp4"
NonV_path = "/content/drive/MyDrive/WVM/Violence/Testingvideos/nonv.mp4"
V_test = "/content/drive/MyDrive/WVM/Violence/Testingvideos/Testing video.mp4"    

# **Mob**

In [8]:
# initialize minimum probability to filter weak detections along with
# the threshold when applying non-maxima suppression
MIN_CONF = 0.3
NMS_THRESH = 0.3

# define the minimum safe distance (in pixels) that two people can be
# from each other
MIN_DISTANCE = 50

In [9]:
# import the necessary packages

import numpy as np
import cv2

def detect_people(frame, net, ln, personIdx=0):
	# grab the dimensions of the frame and  initialize the list of
	# results
	(H, W) = frame.shape[:2]
	results = []

	# construct a blob from the input frame and then perform a forward
	# pass of the YOLO object detector, giving us our bounding boxes
	# and associated probabilities
	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),
		swapRB=True, crop=False)
	net.setInput(blob)
	layerOutputs = net.forward(ln)

	# initialize our lists of detected bounding boxes, centroids, and
	# confidences, respectively
	boxes = []
	centroids = []
	confidences = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability)
			# of the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter detections by (1) ensuring that the object
			# detected was a person and (2) that the minimum
			# confidence is met
			if classID == personIdx and confidence > MIN_CONF:
				# scale the bounding box coordinates back relative to
				# the size of the image, keeping in mind that YOLO
				# actually returns the center (x, y)-coordinates of
				# the bounding box followed by the boxes' width and
				# height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top
				# and and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates,
				# centroids, and confidences
				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	# apply non-maxima suppression to suppress weak, overlapping
	# bounding boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, MIN_CONF, NMS_THRESH)



	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			# update our results list to consist of the person
			# prediction probability, bounding box coordinates,
			# and the centroid
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	# return the list of results
	return results

In [10]:
# USAGE
# python social_distance_detector.py --input pedestrians.mp4
# python social_distance_detector.py --input pedestrians.mp4 --output output.avi

# import the necessary packages
from google.colab.patches import cv2_imshow
from scipy.spatial import distance as dist
import numpy as np
import argparse
import imutils
import cv2
import os

def detect_mob():
  # construct the argument parse and parse the arguments
  ap = argparse.ArgumentParser()
  ap.add_argument("-i", "--input", type=str, default="",
      help="path to (optional) input video file")
  ap.add_argument("-o", "--output", type=str, default="",
      help="path to (optional) output video file")
  ap.add_argument("-d", "--display", type=int, default=1,
      help="whether or not output frame should be displayed")
  args = vars(ap.parse_args(["--input","/content/drive/MyDrive/WVM/Mob/Mob-detection/Raw footage as Paris' Yellow Vest protest turns violent.mp4","--output","my_output.avi","--display","500"]))

  # load the COCO class labels our YOLO model was trained on
  labelsPath = os.path.sep.join(["/content/drive/MyDrive/WVM/Mob/Mob-detection/yolo-coco/coco.names"])
  LABELS = open(labelsPath).read().strip().split("\n")

  # derive the paths to the YOLO weights and model configuration
  weightsPath = os.path.sep.join(["/content/drive/MyDrive/WVM/Mob/Mob-detection/yolo-coco/yolov3.weights"])
  configPath = os.path.sep.join(["/content/drive/MyDrive/WVM/Mob/Mob-detection/yolo-coco/yolov3.cfg"])

  # load our YOLO object detector trained on COCO dataset (80 classes)
  print("[INFO] loading YOLO from disk...")
  net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

  # determine only the *output* layer names that we need from YOLO
  ln = net.getLayerNames()
  ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]

  # initialize the video stream and pointer to output video file
  print("[INFO] accessing video stream...")
  vs = cv2.VideoCapture(args["input"] if args["input"] else 0)
  writer = None
  cnt=0
  # loop over the frames from the video stream
  while cnt<40:
      # read the next frame from the file
      (grabbed, frame) = vs.read()
      cnt=cnt+1
      print(cnt)

      # if the frame was not grabbed, then we have reached the end
      # of the stream
      if not grabbed:
          break

      # resize the frame and then detect people (and only people) in it
      frame = imutils.resize(frame, width=700)
      results = detect_people(frame, net, ln,
      personIdx=LABELS.index("person"))                                                                                                  

      # initialize the set of indexes that violate the minimum social
      # distance
      violate = set()

      # ensure there are *at least* five people detections (required in
      # order to compute our pairwise distance maps)
      if len(results) >= 1:
          # extract all centroids from the results and compute the
          # Euclidean distances between all pairs of the centroids
          centroids = np.array([r[2] for r in results])
          D = dist.cdist(centroids, centroids, metric="euclidean")

          # loop over the upper triangular of the distance matrix
          for i in range(0, D.shape[0]):
              for j in range(i + 1, D.shape[1]):
                  # check to see if the distance between any two
                  # centroid pairs is less than the configured number
                  # of pixels
                  if D[i, j] < MIN_DISTANCE:
                      # update our violation set with the indexes of
                      # the centroid pairs
                      violate.add(i)
                      violate.add(j)

      # loop over the results
          for (i, (prob, bbox, centroid)) in enumerate(results):
              # extract the bounding box and centroid coordinates, then
              # initialize the color of the annotation
              (startX, startY, endX, endY) = bbox
              (cX, cY) = centroid
              color = (0, 255, 0)

              # if the index pair exists within the violation set, then
              # update the color
              if i in violate:
                  color = (0, 0, 255)

              # draw (1) a bounding box around the person and (2) the
              # centroid coordinates of the person,
              cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
              cv2.circle(frame, (cX, cY), 5, color, 1)

      # draw the total number of social distancing violations on the
      # output frame
      text = "Mob Detected: {}".format(len(violate))
      cv2.putText(frame, text, (10, frame.shape[0] - 25),
          cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 255), 3)

      key = cv2.waitKey(1) & 0xFF

          # if the `q` key was pressed, break from the loop
      if key == ord("q"):
          break

      # if an output video file path has been supplied and the video
      # writer has not been initialized, do so now
      if args["output"] != "" and writer is None:
          # initialize our video writer
          push_mob()
          fourcc = cv2.VideoWriter_fourcc(*"MJPG")
          writer = cv2.VideoWriter(args["output"], fourcc, 25,
              (frame.shape[1], frame.shape[0]), True)

      # if the video writer is not None, write the frame to the output
      # video file
      if writer is not None:
          writer.write(frame)

In [11]:
import multiprocessing as mp

In [12]:
functions=[detect_mob()]
processes=[]
for functions in functions:
    process = mp.Process(target=functions)
    processes.append(process)
    process.start()

# Wait for each process to complete
for process in processes:
    process.join()

print('All functions have completed.')

[INFO] loading YOLO from disk...
[INFO] accessing video stream...
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
All functions have completed.
